In [18]:
# imports 
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
from tabulate import tabulate
import mysql.connector
from datetime import datetime

db = mysql.connector.connect(
    host="localhost",    
    user="root",          
    password="_Joseph344", 
    database="NFL_Project2" 
)
cursor = db.cursor()

In [19]:
# Set up webdriver 

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# navigate to the ESPN NFL scoreboard page
driver.get("https://www.espn.com/nfl/scoreboard")
time.sleep(5)  # Allow page to load

In [20]:
# Initialize WebDriverWait
wait = WebDriverWait(driver, 15)  

In [ ]:
# start webdriver 
wait = WebDriverWait(driver, 15)  

all_links = []

while True:
    # Use javascript to get all elements 
    elements = driver.execute_script(
        "return [...document.querySelectorAll('.Arrow.flex.justify-center.items-center.Arrow--left')]"
    )

    # check if any elements were found
    if not elements:
        print("No elements found.")
        break

    # get the href attributes using javascript
    week_js1 = driver.execute_script(
        "return [...document.querySelectorAll('div.Week.currentWeek div.Week__wrapper div.custom--week a')].map(e => e.href);"
    )


    # filter for only links containing '/season/2'
    filtered_links = [link for link in week_js1 if "/seasontype/2" in link]
    
    # add the extracted links to the list
    all_links.extend(filtered_links)

    # use javascript to check if the button is disabled
    is_disabled = driver.execute_script(
        """
        return document.querySelector('.Arrow.flex.justify-center.items-center.Arrow--left').classList.contains('disabled');
        """
    )

    # if the button is disabled, stop the loop
    if is_disabled:
        print("Reached the beginning of the carousel. Stopping.")
        break


    elements[0].click()

# print all the collected links after the loop finishes
print("All links collected:")
for link in all_links:
    print(link)

In [ ]:
boxscore_links = []

# loop through each week link
for link in all_links:
    try:
        print(f"\nNavigating to: {link}")
        driver.get(link)  # Open the page in the browser


        wait.until(EC.presence_of_element_located((By.ID, "fittPageContainer")))


        # extract the week number from the URL
        week_number = link.split("/week/")[1].split("/")[0]  # Extracts the "18" from "/week/18/"

        # extract boxscore links
        boxscores = driver.find_elements(By.XPATH, '//a[contains(@href, "/boxscore/")]')

        for item in boxscores:
            boxscore = item.get_attribute('href')
            if boxscore and '/boxscore/' in boxscore:
                boxscore_links.append(boxscore)  # Append only the link (no dictionary)
                print(f"Found boxscore link: {boxscore}")

        if weeks_processed >= max_weeks_to_process:
            break  

    except Exception as e:
        print(f"Error processing {link}: {e}")


print("\nAll boxscore links collected:")
for link in boxscore_links:
    game_ids = [link.split("gameId/")[-1] for link in boxscore_links]
    print(game_ids)

In [23]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager


try:
    for links1 in boxscore_links:

        # open the page
        driver.get(links1)

 
        try:
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//div[contains(@class, "GameInfo__BettingItem")]'))
            )
        except:
            print(f"Skipping {links1} - Betting data not found.")
            continue  # Skip this game if betting data is missing

        # getting the spread
        spread = driver.find_elements(By.XPATH, '//div[contains(@class, "GameInfo__BettingItem")]')

        spread_text = spread[0].text.strip()
        over_text = spread[1].text.strip()


        game_id = links1.split("gameId/")[-1]

  
        game_insert_query = """
        INSERT INTO games (over_text, spread_text, game_id)
        VALUES (%s, %s, %s)
        """
        game_values = (over_text, spread_text, game_id)

        cursor.execute(game_insert_query, game_values)  
        db.commit()

  

except Exception as e:
    print("Error:", e)

finally:
 
    cursor.close()
    db.close()
    driver.quit()












Skipping https://www.espn.com/nfl/boxscore/_/gameId/401671668 - Betting data not found.
Skipping https://www.espn.com/nfl/boxscore/_/gameId/401671668 - Betting data not found.
